<a href="https://colab.research.google.com/github/renanpyd/Course_Google_Colab/blob/main/site/pt-br/tutorials/load_data/csv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2019 The TensorFlow Authors.


In [1]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Carregar dados CSV

Este tutorial fornece um exemplo de como carregar dados CSV de um arquivo em um `tf.data.Dataset`.

Os dados usados neste tutorial foram retirados da lista de passageiros do Titanic. O modelo preverá a probabilidade de sobrevivência de um passageiro com base em características como idade, sexo, classe de passagem e se a pessoa estava viajando sozinha.

## Setup

In [2]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass


In [3]:
from __future__ import absolute_import, division, print_function, unicode_literals
import functools

import numpy as np
import tensorflow as tf

In [4]:
TRAIN_DATA_URL = "https://storage.googleapis.com/tf-datasets/titanic/train.csv"
TEST_DATA_URL = "https://storage.googleapis.com/tf-datasets/titanic/eval.csv"

train_file_path = tf.keras.utils.get_file("train.csv", TRAIN_DATA_URL)
test_file_path = tf.keras.utils.get_file("eval.csv", TEST_DATA_URL)

24576/13049 [========================================================] - 0s 0us/step


In [5]:
# Facilitar a leitura de valores numpy.
np.set_printoptions(precision=3, suppress=True)

## Carregar dados

Para começar, vejamos a parte superior do arquivo CSV para ver como ele está formatado.

In [6]:
!head {train_file_path}

survived,sex,age,n_siblings_spouses,parch,fare,class,deck,embark_town,alone
0,male,22.0,1,0,7.25,Third,unknown,Southampton,n
1,female,38.0,1,0,71.2833,First,C,Cherbourg,n
1,female,26.0,0,0,7.925,Third,unknown,Southampton,y
1,female,35.0,1,0,53.1,First,C,Southampton,n
0,male,28.0,0,0,8.4583,Third,unknown,Queenstown,y
0,male,2.0,3,1,21.075,Third,unknown,Southampton,n
1,female,27.0,0,2,11.1333,Third,unknown,Southampton,n
1,female,14.0,1,0,30.0708,Second,unknown,Cherbourg,n
1,female,4.0,1,1,16.7,Third,G,Southampton,n


Você pode [carregar isso usando pandas] (pandas.ipynb) e passar as matrizes NumPy para o TensorFlow. Se você precisar escalar até um grande conjunto de arquivos ou precisar de um carregador que se integre ao [TensorFlow e tf.data] (../../guide/data.ipynb), use o `tf.data.experimental. função make_csv_dataset`:

A única coluna que você precisa identificar explicitamente é aquela com o valor que o modelo pretende prever. 

In [7]:
LABEL_COLUMN = 'survived'
LABELS = [0, 1]

Now read the CSV data from the file and create a dataset. 

(For the full documentation, see `tf.data.experimental.make_csv_dataset`)


In [8]:
def get_dataset(file_path, **kwargs):
  dataset = tf.data.experimental.make_csv_dataset(
      file_path,
      batch_size=5, # Artificialmente pequeno para facilitar a exibição de exemplos
      label_name=LABEL_COLUMN,
      na_value="?",
      num_epochs=1,
      ignore_errors=True, 
      **kwargs)
  return dataset

raw_train_data = get_dataset(train_file_path)
raw_test_data = get_dataset(test_file_path)

In [9]:
def show_batch(dataset):
  for batch, label in dataset.take(1):
    for key, value in batch.items():
      print("{:20s}: {}".format(key,value.numpy()))

Cada item do conjunto de dados é um lote, representado como uma tupla de (* muitos exemplos *, * muitos rótulos *). Os dados dos exemplos são organizados em tensores baseados em colunas (em vez de tensores baseados em linhas), cada um com tantos elementos quanto o tamanho do lote (5 neste caso).

Pode ajudar a ver isso por si mesmo.

In [10]:
show_batch(raw_train_data)

sex                 : [b'male' b'male' b'female' b'male' b'female']
age                 : [43. 28. 11. 20. 24.]
n_siblings_spouses  : [0 0 4 0 0]
parch               : [0 0 2 0 0]
fare                : [ 8.05   7.896 31.275  8.05  49.504]
class               : [b'Third' b'Third' b'Third' b'Third' b'First']
deck                : [b'unknown' b'unknown' b'unknown' b'unknown' b'C']
embark_town         : [b'Southampton' b'Cherbourg' b'Southampton' b'Southampton' b'Cherbourg']
alone               : [b'y' b'y' b'n' b'y' b'y']


Como você pode ver, as colunas no CSV são nomeadas. O construtor do conjunto de dados selecionará esses nomes automaticamente. Se o arquivo com o qual você está trabalhando não contém os nomes das colunas na primeira linha, passe-os em uma lista de strings para o argumento `column_names` na função `make_csv_dataset`.

In [11]:
CSV_COLUMNS = ['survived', 'sex', 'age', 'n_siblings_spouses', 'parch', 'fare', 'class', 'deck', 'embark_town', 'alone']

temp_dataset = get_dataset(train_file_path, column_names=CSV_COLUMNS)

show_batch(temp_dataset)

sex                 : [b'male' b'male' b'female' b'female' b'female']
age                 : [24. 28. 24.  5. 14.]
n_siblings_spouses  : [2 0 0 0 1]
parch               : [0 0 2 0 0]
fare                : [73.5    7.25  16.7   12.475 30.071]
class               : [b'Second' b'Third' b'Third' b'Third' b'Second']
deck                : [b'unknown' b'unknown' b'G' b'unknown' b'unknown']
embark_town         : [b'Southampton' b'Southampton' b'Southampton' b'Southampton' b'Cherbourg']
alone               : [b'n' b'y' b'n' b'y' b'n']


Este exemplo vai usar todas as colunas disponíveis. Se você precisar omitir algumas colunas do conjunto de dados, crie uma lista apenas das colunas que planeja usar e passe-a para o argumento (opcional) `select_columns` do construtor.

In [12]:
SELECT_COLUMNS = ['survived', 'age', 'n_siblings_spouses', 'class', 'deck', 'alone']

temp_dataset = get_dataset(train_file_path, select_columns=SELECT_COLUMNS)

show_batch(temp_dataset)

age                 : [28. 11. 30. 58. 28.]
n_siblings_spouses  : [0 0 0 0 0]
class               : [b'Third' b'Third' b'First' b'First' b'First']
deck                : [b'unknown' b'unknown' b'B' b'B' b'unknown']
alone               : [b'y' b'y' b'y' b'y' b'y']


## Pré-processamento dos Dados

Um arquivo CSV pode conter uma variedade de tipos de dados. Normalmente, você deseja converter desses tipos mistos em um vetor de comprimento fixo antes de alimentar os dados em seu modelo.

O TensorFlow possui um sistema interno para descrever conversões de entrada comuns: `tf.feature_column`, consulte [este tutorial] (../keras/feature_columns) para detalhes.


Você pode pré-processar seus dados usando qualquer ferramenta que desejar (como [nltk] (https://www.nltk.org/) ou [sklearn] (https://scikit-learn.org/stable/)) e apenas passar a saída processada para o TensorFlow.


A principal vantagem de fazer o pré-processamento dentro do seu modelo é que, quando você exporta o modelo, ele inclui o pré-processamento. Dessa forma, você pode passar os dados brutos diretamente para o seu modelo.

### Dados contínuos

Se seus dados já estiverem em um formato numérico apropriado, você poderá compactá-los em um vetor antes de transmiti-los ao modelo:

In [13]:
SELECT_COLUMNS = ['survived', 'age', 'n_siblings_spouses', 'parch', 'fare']
DEFAULTS = [0, 0.0, 0.0, 0.0, 0.0]
temp_dataset = get_dataset(train_file_path, 
                           select_columns=SELECT_COLUMNS,
                           column_defaults = DEFAULTS)

show_batch(temp_dataset)

age                 : [28.  40.5 25.  21.  16. ]
n_siblings_spouses  : [0. 0. 0. 0. 0.]
parch               : [0. 2. 0. 0. 0.]
fare                : [ 7.896 14.5    7.05   8.663  7.733]


In [14]:
example_batch, labels_batch = next(iter(temp_dataset)) 

Aqui está uma função simples que agrupará todas as colunas:

In [15]:
def pack(features, label):
  return tf.stack(list(features.values()), axis=-1), label

Aplique isso a cada elemento do conjunto de dados:

In [16]:
packed_dataset = temp_dataset.map(pack)

for features, labels in packed_dataset.take(1):
  print(features.numpy())
  print()
  print(labels.numpy())

[[ 51.      0.      1.     61.379]
 [ 44.      0.      0.      7.925]
 [ 49.      1.      0.     56.929]
 [ 16.      0.      0.      8.05 ]
 [ 38.      0.      0.    227.525]]

[0 1 1 0 1]


Se você tiver tipos de dados mistos, poderá separar esses campos numéricos simples. A API `tf.feature_column` pode lidar com eles, mas isso gera alguma sobrecarga e deve ser evitado, a menos que seja realmente necessário. Volte para o conjunto de dados misto:

In [17]:
show_batch(raw_train_data)

sex                 : [b'female' b'female' b'male' b'male' b'male']
age                 : [ 9. 28. 58. 35. 28.]
n_siblings_spouses  : [2 0 0 0 0]
parch               : [2 0 0 0 0]
fare                : [34.375 79.2   29.7   26.55   7.896]
class               : [b'Third' b'First' b'First' b'First' b'Third']
deck                : [b'unknown' b'unknown' b'B' b'unknown' b'unknown']
embark_town         : [b'Southampton' b'Cherbourg' b'Cherbourg' b'Cherbourg' b'Southampton']
alone               : [b'n' b'y' b'y' b'y' b'y']


In [18]:
example_batch, labels_batch = next(iter(temp_dataset)) 

Portanto, defina um pré-processador mais geral que selecione uma lista de recursos numéricos e os agrupe em uma única coluna:

In [19]:
class PackNumericFeatures(object):
  def __init__(self, names):
    self.names = names

  def __call__(self, features, labels):
    numeric_features = [features.pop(name) for name in self.names]
    numeric_features = [tf.cast(feat, tf.float32) for feat in numeric_features]
    numeric_features = tf.stack(numeric_features, axis=-1)
    features['numeric'] = numeric_features

    return features, labels

In [20]:
NUMERIC_FEATURES = ['age','n_siblings_spouses','parch', 'fare']

packed_train_data = raw_train_data.map(
    PackNumericFeatures(NUMERIC_FEATURES))

packed_test_data = raw_test_data.map(
    PackNumericFeatures(NUMERIC_FEATURES))

In [21]:
show_batch(packed_train_data)

sex                 : [b'male' b'male' b'female' b'male' b'male']
class               : [b'Second' b'Third' b'Third' b'Third' b'Third']
deck                : [b'unknown' b'unknown' b'unknown' b'unknown' b'unknown']
embark_town         : [b'Southampton' b'Southampton' b'Southampton' b'Southampton'
 b'Southampton']
alone               : [b'y' b'y' b'y' b'y' b'y']
numeric             : [[50.     0.     0.    13.   ]
 [24.     0.     0.     8.05 ]
 [23.     0.     0.     7.925]
 [19.     0.     0.     0.   ]
 [19.     0.     0.     7.775]]


In [22]:
example_batch, labels_batch = next(iter(packed_train_data)) 

#### Normalização dos dados

Dados contínuos sempre devem ser normalizados.

In [23]:
import pandas as pd
desc = pd.read_csv(train_file_path)[NUMERIC_FEATURES].describe()
desc

,age,n_siblings_spouses,parch,fare
count,627.000000,627.000000,627.000000,627.000000
mean,29.631308,0.545455,0.379585,34.385399
std,12.511818,1.151090,0.792999,54.597730
min,0.750000,0.000000,0.000000,0.000000
25%,23.000000,0.000000,0.000000,7.895800
50%,28.000000,0.000000,0.000000,15.045800
75%,35.000000,1.000000,0.000000,31.387500
max,80.000000,8.000000,5.000000,512.329200


In [24]:
MEAN = np.array(desc.T['mean'])
STD = np.array(desc.T['std'])

In [25]:
def normalize_numeric_data(data, mean, std):
  # Center the data
  return (data-mean)/std


Agora crie uma coluna numérica. A API `tf.feature_columns.numeric_column` aceita um argumento `normalizer_fn`, que será executado em cada lote.

Ligue o `MEAN` e o` STD` ao normalizador fn usando [`functools.partial`] (https://docs.python.org/3/library/functools.html#functools.partial)

In [26]:
# Veja o que você acabou de criar.
normalizer = functools.partial(normalize_numeric_data, mean=MEAN, std=STD)

numeric_column = tf.feature_column.numeric_column('numeric', normalizer_fn=normalizer, shape=[len(NUMERIC_FEATURES)])
numeric_columns = [numeric_column]
numeric_column

NumericColumn(key='numeric', shape=(4,), default_value=None, dtype=tf.float32, normalizer_fn=functools.partial(<function normalize_numeric_data at 0x7f47ee35eef0>, mean=array([29.631,  0.545,  0.38 , 34.385]), std=array([12.512,  1.151,  0.793, 54.598])))

Ao treinar o modelo, inclua esta coluna de característica para selecionar e centralizar este bloco de dados numéricos:

In [27]:
example_batch['numeric']

<tf.Tensor: shape=(5, 4), dtype=float32, numpy=
array([[37.   ,  2.   ,  0.   ,  7.925],
       [22.   ,  0.   ,  0.   ,  7.25 ],
       [48.   ,  1.   ,  0.   , 76.729],
       [32.   ,  0.   ,  0.   ,  8.05 ],
       [62.   ,  0.   ,  0.   , 10.5  ]], dtype=float32)>

In [28]:
numeric_layer = tf.keras.layers.DenseFeatures(numeric_columns)
numeric_layer(example_batch).numpy()

array([[ 0.589,  1.264, -0.479, -0.485],
       [-0.61 , -0.474, -0.479, -0.497],
       [ 1.468,  0.395, -0.479,  0.776],
       [ 0.189, -0.474, -0.479, -0.482],
       [ 2.587, -0.474, -0.479, -0.437]], dtype=float32)

A normalização baseada em média usada aqui requer conhecer os meios de cada coluna antes do tempo.

### Dados categóricos

Algumas das colunas nos dados CSV são colunas categóricas. Ou seja, o conteúdo deve ser um dentre um conjunto limitado de opções.

Use a API `tf.feature_column` para criar uma coleção com uma `tf.feature_column.indicator_column` para cada coluna categórica.


In [29]:
CATEGORIES = {
    'sex': ['male', 'female'],
    'class' : ['First', 'Second', 'Third'],
    'deck' : ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J'],
    'embark_town' : ['Cherbourg', 'Southhampton', 'Queenstown'],
    'alone' : ['y', 'n']
}


In [30]:
categorical_columns = []
for feature, vocab in CATEGORIES.items():
  cat_col = tf.feature_column.categorical_column_with_vocabulary_list(
        key=feature, vocabulary_list=vocab)
  categorical_columns.append(tf.feature_column.indicator_column(cat_col))

In [31]:
# Veja o que você acabou de criar.
categorical_columns

[IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='sex', vocabulary_list=('male', 'female'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='class', vocabulary_list=('First', 'Second', 'Third'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='deck', vocabulary_list=('A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='embark_town', vocabulary_list=('Cherbourg', 'Southhampton', 'Queenstown'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='alone', vocabulary_list=('y', 'n'), dtype=tf.string, default_value=-1, num_oov_buckets=0))]

In [32]:
categorical_layer = tf.keras.layers.DenseFeatures(categorical_columns)
print(categorical_layer(example_batch).numpy()[0])

[0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]


Isso fará parte de uma entrada de processamento de dados posteriormente, quando você construir o modelo.

### Camada combinada de pré-processamento

Adicione as duas coleções de colunas de recursos e passe-as para um `tf.keras.layers.DenseFeatures` para criar uma camada de entrada que extrairá e pré-processará os dois tipos de entrada:

In [33]:
preprocessing_layer = tf.keras.layers.DenseFeatures(categorical_columns+numeric_columns)

In [34]:
print(preprocessing_layer(example_batch).numpy()[0])

[ 0.     1.     0.     0.     1.     0.     0.     0.     0.     0.
  0.     0.     0.     0.     0.     0.     0.     0.     0.589  1.264
 -0.479 -0.485  1.     0.   ]


## Construir o modelo

Crie um `tf.keras.Sequential`, começando com o `preprocessing_layer`.

In [35]:
model = tf.keras.Sequential([
  preprocessing_layer,
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(1),
])

model.compile(
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
    optimizer='adam',
    metrics=['accuracy'])

## Treinar, avaliar, e prever

Agora o modelo pode ser instanciado e treinado.

In [36]:
train_data = packed_train_data.shuffle(500)
test_data = packed_test_data

In [37]:
model.fit(train_data, epochs=20)

Epoch 1/20
Consider rewriting this model with the Functional API.
Consider rewriting this model with the Functional API.
126/126 [==============================] - 2s 6ms/step - loss: 0.4982 - accuracy: 0.7337
Epoch 2/20
126/126 [==============================] - 0s 3ms/step - loss: 0.4155 - accuracy: 0.8198
Epoch 3/20
126/126 [==============================] - 0s 3ms/step - loss: 0.4050 - accuracy: 0.8214
Epoch 4/20
126/126 [==============================] - 0s 3ms/step - loss: 0.3924 - accuracy: 0.8325
Epoch 5/20
126/126 [==============================] - 0s 3ms/step - loss: 0.3791 - accuracy: 0.8309
Epoch 6/20
126/126 [==============================] - 0s 3ms/step - loss: 0.3694 - accuracy: 0.8389
Epoch 7/20
126/126 [==============================] - 0s 3ms/step - loss: 0.3605 - accuracy: 0.8469
Epoch 8/20
126/126 [==============================] - 0s 3ms/step - loss: 0.3588 - accuracy: 0.8373
Epoch 9/20
126/126 [==============================] - 0s 3ms/step - loss: 0.3496 - accurac

Depois que o modelo é treinado, você pode verificar sua acurácia no conjunto `test_data`.

In [38]:
test_loss, test_accuracy = model.evaluate(test_data)

print('\n\nTest Loss {}, Test Accuracy {}'.format(test_loss, test_accuracy))

Consider rewriting this model with the Functional API.
53/53 [==============================] - 1s 3ms/step - loss: 0.4342 - accuracy: 0.8447


Test Loss 0.4341820180416107, Test Accuracy 0.8446969985961914


Use `tf.keras.Model.predict` para inferir rótulos em um lote ou em um conjunto de dados de lotes.

In [39]:
predictions = model.predict(test_data)

# Mostrar alguns resultados
for prediction, survived in zip(predictions[:10], list(test_data)[0][1][:10]):
  print("Predicted survival: {:.2%}".format(prediction[0]),
        " | Actual outcome: ",
        ("SURVIVED" if bool(survived) else "DIED"))


Consider rewriting this model with the Functional API.
Predicted survival: -235.39%  | Actual outcome:  SURVIVED
Predicted survival: -187.53%  | Actual outcome:  DIED
Predicted survival: 175.69%  | Actual outcome:  DIED
Predicted survival: 594.40%  | Actual outcome:  DIED
Predicted survival: -214.19%  | Actual outcome:  DIED
